In [45]:
import numpy as np
import time

In [58]:
# global paremeters

N1 = 10
N2 = 10
l1 = 1
l2 = 1
h1 = l1 / (N1)
h2 = l2 / (N2)

eps = 0.0001

iterations = [10, 20, 30]

# my functions
def u_a(x, y):
    return x * (x - 1) * y * (y - 1)
    #return x * (x - 2) * y * (y - 2)
    #return np.sin(np.pi * x) * np.sin(np.pi * y)

def b(x, y):
    return -2 * ((y**2 - y) * (3 * x**2 + y**2 - x + 1) + x**2 - x)
    #return np.pi * np.sin(np.pi * y) * (-2 * x * np.cos(np.pi * x) + (2 + x**2 + y**2) * np.pi * np.sin(np.pi * x))

In [59]:
def get_mu_k2(r, sin_k2_j, sin_k1_i):
    mu = np.zeros((N1 + 1, N2 + 1))
    for i in range(1,N1):
        for k2 in range(1, N2):
            s = 0
            for j in range(1, N2):
                s += r[i][j] * sin_k2_j[k2, j]
            mu[i, k2] = s
    return mu

def mu_k2(i, k2, g_mu):

    return g_mu[i,k2]

def mu_k1k2(k1, k2, g_mu, sin_k2_j, sin_k1_i):
    s = 0
    for i in range(1, N1):
        s += mu_k2(i, k2, g_mu) * sin_k1_i[k1, i]
    return s

def v_k2(i, k2, g_mu, sob, sin_k2_j, sin_k1_i):
    s = 0
    for k1 in range(1, N1):
        s += mu_k1k2(k1, k2, g_mu, sin_k2_j, sin_k1_i) * sob[k1, k2] * sin_k1_i[k1 ,i]
    return s

def sobstv_ch(k1, k2):
    return (4. / h1**2) * np.sin(k1 * np.pi * h1 / (2 * l1))**2 + (4 / h2**2) * np.sin(k2 * np.pi * h2 / (2 * l2))**2

def v(i, j, g_mu, sob, sin_k2_j, sin_k1_i):
    s = 0
    for k2 in range(1,N2):
        s += v_k2(i, k2, g_mu, sob, sin_k2_j, sin_k1_i) * sin_k2_j[k2 ,j]
    return (4 / (N1 * N2)) * s

def k11(x,y):
    return 1 + x**2 + y**2

def lambda_xx(vn, i, j):
    return (1 / h1**2) * (vn[i+1][j] * 0.5 * (k11(i * h1, j * h2) + k11((i + 1) * h1, j * h2)) - vn[i][j]* 0.5 *( 2 * k11(i * h1, j * h2) + k11((i+1)*h1, j*h2)+ k11((i-1)*h1, j*h2) ) + vn[i-1][j]*0.5*(k11(i*h1, j*h2) + k11((i-1)*h1, j*h2)))

def lambda_yy(vn, i, j):
    return (1 / h2**2) * (vn[i][j+1] - vn[i][j] * 2 + vn[i][j-1])

def A(vn):
    A_vn = np.zeros((N1+1,N2+1))
    for i in range(1, N1):
        for j in range(1, N2):
            A_vn[i][j] = - (lambda_xx(vn, i, j) + lambda_yy(vn, i, j))
    return A_vn

def A_i(vn, i, j):
    A_vn = - (lambda_xx(vn, i, j) + lambda_yy(vn, i, j) )
    return A_vn

def alpha(n):
    c1 = 1
    c2 = 3
    alp = np.zeros(n)
    for i in range(0,n):
        alp[i] = 1 / ((c1 + c2) * 0.5 + (c2 - c1) * 0.5 * np.cos(np.pi * (2 * (i + 1) - 1) * 0.5 / n))

    return alp

def full_sobstv_ch():
    sob = np.zeros((N1, N2))
    for k1 in range(1, N1):
        for k2 in range(1, N2):
            sob[k1][k2] = 1 / ((4. / h1**2) * np.sin(k1 * np.pi * h1 / (2 * l1))**2 + (4 / h2**2) * np.sin(k2 * np.pi * h2 / (2 * l2))**2)
    return sob

def get_sin_k1_i():
    sin1 = np.zeros((N1+1, N1+1))
    for i in range(1, N1):
        for k1 in range(1, N1):
            sin1[i, k1] = np.sin(k1 * np.pi * i / N1)
    return sin1

def get_sin_k2_j():
    sin1 = np.zeros((N2+1, N2+1))
    for j in range(1, N2):
        for k2 in range(1, N2):
            sin1[j, k2] = np.sin(k2 * np.pi * j / N2)
    return sin1


In [61]:
sobs = full_sobstv_ch()
sin_k2_j = get_sin_k2_j()
sin_k1_i = get_sin_k1_i()

for iter in iterations:

    v_0 = np.zeros((N1+1, N2+1))
    v_n = v_0
    x=np.linspace(0,l1, N1+1)
    y=np.linspace(0,l2, N2+1)
    X,Y=np.meshgrid(x, y)
    b_mesh = np.transpose(b(X, Y))
    b_mesh[:, [0, -1]] = 0
    b_mesh[[0, -1], :] = 0

    u_a_mesh = np.transpose(u_a(X, Y))
    A_n = A(v_n)
    error = np.max(np.abs(A_n - b_mesh))

    step = 0
    alp = alpha(iter)
    print(f'Iterations: {iter}')

    while(error > eps and step != iter):
        r_n =  A_n - b_mesh
        g_mu = get_mu_k2(r_n, sin_k2_j, sin_k1_i)
        #start = time.time()
        y = np.zeros((N1+1, N2+1))
        for i in range(1, N1):
            for j in range(1, N2):
                y[i][j] = v(i, j, g_mu, sobs, sin_k1_i, sin_k2_j)
        #end = time.time() - start
        #print(end)
        print(f'||r_n||_inf = {error} ')

        v_n -= alp[step] * y
        A_n = A(v_n)
        error = np.max(np.abs(A_n - b_mesh))
        step += 1

    print('\n')
    print(f'||v_n-u_a||_inf = {np.max(np.abs(v_n - u_a_mesh))}')
    print('\n')

Iterations: 10
||r_n||_inf = 1.57 
||r_n||_inf = 0.7447029613558729 
||r_n||_inf = 0.42460693257674653 
||r_n||_inf = 0.24227116697199036 
||r_n||_inf = 0.13078750493334834 
||r_n||_inf = 0.06574213222004399 
||r_n||_inf = 0.027503835828145307 
||r_n||_inf = 0.008893106109724824 
||r_n||_inf = 0.001876785860144503 
||r_n||_inf = 0.00015152197567436243 


||v_n-u_a||_inf = 0.00011996575531486586


Iterations: 20
||r_n||_inf = 1.57 
||r_n||_inf = 0.7465515733239252 
||r_n||_inf = 0.4333256607750433 
||r_n||_inf = 0.26040532862705834 
||r_n||_inf = 0.15580517875861477 
||r_n||_inf = 0.09406949564997347 
||r_n||_inf = 0.056207788280937754 
||r_n||_inf = 0.032325441682723666 
||r_n||_inf = 0.017724063788352784 
||r_n||_inf = 0.009383112130542104 
||r_n||_inf = 0.004716479809381502 
||r_n||_inf = 0.00221491844130961 
||r_n||_inf = 0.0009341459861184065 
||r_n||_inf = 0.0003428987407024797 
||r_n||_inf = 0.0001048775563090576 


||v_n-u_a||_inf = 0.00012005029655598465


Iterations: 30
||r_n|

In [ ]:
N1 = 10
N2 = 15
l1 = 1
l2 = 1
h1 = l1 / (N1)
h2 = l2 / (N2)

iters = 20000

def get_B(v_):
    B = np.zeros((N1+1, N2+1))
    for i in range(0,N1 - 1):
        for j in range(0, N2 - 1):
            B[i, j] = (-v_[i+1,j] + 2*v_[i,j] - v_[i-1,j])/(h1*h1) + (-v_[i,j+1] + 2*v_[i,j] - v_[i,j-1])/(h2*h2)
    return B

def scal(v_1, v_2):
    s = 0
    for i in range(0, N1):
        for j in range(0, N2):
            s += v_1[i,j]*v_2[i,j]
    return s * h1 * h2
fal = 0
for i in range(0, iters):
    v_rand = np.random.rand(N1+1, N2+1)
    v_rand[:, [0, -1]] = 0
    v_rand[[0, -1], :] = 0
    B = get_B(v_rand)
    A_ = A(v_rand)
    sk_1 = scal(B,v_rand)
    sk_2 = scal(A_, v_rand)
    if not (sk_1 <= sk_2 <= 3*sk_1):
        fal += 1
fal

0